In [4]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
#from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

from modelnet40_pi import ModelNet40PI

%matplotlib inline

In [6]:
train_data = ModelNet40PI(
    root="/home/mishal2/height-persist/features/red_ModelNet40",
    train=True
)

test_data = ModelNet40PI(
    root="/home/mishal2/height-persist/features/red_ModelNet40",
    train=False
)


train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=2, shuffle=True)

In [63]:
#torch.set_default_dtype(torch.float)
from torch import nn

class ConvNet(nn.Module):
    '''
    Simple Convolutional Neural Network
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(40, 50, kernel_size=3),
            nn.MaxPool2d(kernel_size = 2),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(50, 20, kernel_size=3),
            nn.MaxPool2d(kernel_size = 2),
            nn.ReLU(),
            nn.Dropout(p=0.2)  ,
            nn.Flatten(),
            nn.Linear(20*3*3, 100),
            nn.ReLU(),
            nn.Dropout(p=0.2),  
            nn.Linear(100, 40),
            nn.Softmax(dim=1)     
            )

    def forward(self, x):
        #x = x.view(x.shape[0], -1)
        return self.layers(x)


In [36]:
loss_function = nn.CrossEntropyLoss()

k_folds = 3
num_epochs = 1
kfold = KFold(n_splits=k_folds, shuffle=True)
  
dataset = train_data


In [64]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=1, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler)
    
    network = ConvNet()
    network.apply(reset_weights)
    network = network.double()
    
    optimizer = torch.optim.Adam(network.parameters())
    
    for epoch in range(0, num_epochs):

      print(f'Starting epoch {epoch+1}')

      current_loss = 0.0

      for i, data in enumerate(trainloader, 0):
        
        inputs, targets = data
        optimizer.zero_grad()
        outputs = network(inputs)          
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        if i % 100 == 99:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 100))
            current_loss = 0.0
            
    print('Training process has finished. Saving trained model.')
    print('Starting testing')
    
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    correct, total = 0, 0
    with torch.no_grad():

      for i, data in enumerate(testloader, 0):

        inputs, targets = data
        outputs = network(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)
    
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average: {sum/len(results.items())} %')



FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(40, 50, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(50, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=180, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=100, out_features=40, bias=True)
Starting epoch 1
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1, 40, 20, 20])
torch.Size([1, 40])
torch.float64
torch.Size([1,

KeyboardInterrupt: 